<a href="https://csdms.colorado.edu"><img style="float: center; width: 75%" src="https://raw.githubusercontent.com/csdms/ivy/main/media/logo.png"></a>

# Advection

Advection is a transport mechanism of a substance or conserved property---temperature, moisture, aerosols---through the bulk motion of a fluid.

The advection equation has an analytical solution (see below), so why discretise?
Because not all forms of the analytical solution are known;
in fact, most of the time, an analytical solution is not known,
especially in the earth sciences, where sets of simultaneous, nonlinear, partial differential equations have to be solved.
To solve such equations in a coupled system, numerical methods are required.

In this notebook, we'll use a finite difference scheme based the simplest form of a truncated Taylor series expansion.
While alternative methods are available to solve PDEs,
many models in the geosciences use finite difference methods
because of their conceptual simplicity and ease of coding
(Prof. N. Van Lipzig, personal communication).

Author: Prof. Benjamin Campforts

## The advection equation

A conceptually very simple PDE is the Advection Equation: 

$$\frac{\partial C}{\partial t} = -v \frac{\partial C}{\partial x}
\label{eq:1}\tag{1}$$
or 
$$\frac{\partial C}{\partial t} +v \frac{\partial C}{\partial x}=0
\label{1b}$$

where $C$ is the aerosol concentration and $v$ is a constant wind speed at which aerosol concentrations are advected. The equation above is a prototype of an **initial value problem**: The solution is obtained by using the known initial values and marching or advancing in time. The solution of this equation can be obtained directly from the initial conditions:

$$ C(x,t) = C(x-vt,0) \label{eq:2}\tag{2}$$

In the following we will discretize Eq. $\ref{eq:1}$ using a first order, finite difference upwind scheme. 

All finite difference methods can be represented with a numerical stencil:

<img src="./media/Stencil1.png" style="width:3in;height:1in" />

For the upwind scheme in particular, we only need three nodes: 
<img src="./media/Stencil2.png" style="width:3in;height:1.25in" />

**BUT** the nodes you need to consider depend on the sign of the velocity: 
<img src="./media/UpwindScheme.png" style="width:6.5in;height:3.76in" />

If $v$ is positive, than the direction of the advected flow is from left to right along the $x$ axis. To capture such a positive flow, the upwind discretisation scheme uses the info from the neighbouring node in the upwind direction, that is, the node to the left, indicated by $j-1$ on the sketch. In this case the the Finite Difference Equation (FDE) of Eq. $\ref{eq:1}$ can be written as:

$$\frac{\tilde{C}_j^{n+1} - \tilde{C}_j^n}{\Delta t} + v
\frac{\tilde{C}_j^n - \tilde{C}_{j-1}^n}{\Delta x} = 0
\label{eq:3}\tag{3}$$

If however, the velocity is negative, the FDE becomes: 

$$\frac{\tilde{C}_j^{n+1} - \tilde{C}_j^n}{\Delta t} + v
\frac{\tilde{C}_{j+1}^n - \tilde{C}_j^n}{\Delta x} = 0
\label{eq:4}\tag{4}$$

To obtain this equation, we have taken discrete values for $x$ and $t$: $x_j=j\Delta x$, $t_n = n \Delta t$. The solution of the finite difference equation is also defined at the discrete points ($j\Delta x$,$n \Delta t$): $\tilde{C}_j^n=\tilde{C}(j\Delta x,n \Delta t)$. Note that we used $C$ to denote the solution of the PDE
(continuous) and $\tilde{C}$ to denote the solution of the finite difference equation (FDE), a discrete solution. In the remainder of the notebook, we will use the symbol $C$ for both the analytical and numerical solution.


## Numerical solutions

In the following examples,
we'll explore numerical solutions to the advection equation.

We'll use NumPy for data and Matplotlib for visualization.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

First, develop some code that will represent a perturbation of some quantity that can be advected.
Here, we'll call it temperature,
but it could be any other passive scalar, like concentration of a pollutant.

In [ ]:
# Initialize constants
dx = 500  # spatial step (m)
wavelength = 5e4  # wavelength (m-1)
Lx = 25e4  # size of the domain (m)
amplitude = 5  # wave amplitude (K)

# Set up x-axis values
x = np.arange(start=0, stop=Lx, step=dx)

# Create truncated sine wave from x-axis values
b = x <= wavelength
c = x > wavelength
T = np.zeros_like(x)
T[b] = amplitude * np.sin(x[b] * 2 * np.pi / wavelength)
T[c] = 0

# Plot the graph
plt.figure()
plt.plot(x / 1.0e3, T, "--k")
plt.ylim((-6, 6))
plt.xlim((0, max(x / 1e3)))
plt.xlabel("x (km)")
plt.ylabel("T (K)")
plt.title("Temperature Perturbation")
plt.show()

Try modifying with the parameters (e.g., amplitude and wavelength) to better understand their meaning.

### Upwind solution

Now, lets solve the advection equation using the upwind FDE. You can use either `for` loops or vector operations (NumPy arrays) to iterate over the temperature vector in space. Start with the solution for positive velocity (that is, solve Eq. 3) and assume: 

- Wind speed `v = 10` m/s
- Grid spacing `dx = 500` m (see previous code block)
- Total model run duration = 3 hours
- Calculate time step `dt` using the CFL criterion: `dt = 0.9 * dx / abs(v)`
- Periodic boundary conditions: the solution at the left boundary depends on the solution at the right boundary if velocity is positive and vice versa for negative velocities. 
- Plot the wave every 10^5 iterations 

The structure of the code should look like this: 

~~~
# Set parameter values
v = 
run_duration = 
dt = 

# Initialize temperature array 
T = 
T[:] = 

# Calculate numerical solution
iter = 
while iter*dt < total_time:
    iter += 1
    if v > 0:
        # here comes the solution of the advection equation  
    
    if iter % 10 == 0: 
        plt.plot(x/1.e3, T, 'b') 
        plt.xlim((0, max(x/1e3)))
        plt.xlabel('x (km)')
        plt.ylabel('T (K)')
        plt.title("Advection of Temperature Perturbation")
        plt.show()
~~~

In [ ]:
# Set parameter values
v = 10
run_duration = 3 * 60 * 60.0
dt = 0.9 * dx / abs(v)
print(f"Time step = {dt} s")

In [ ]:
# Initialize temperature array
T = np.zeros_like(x)
T[b] = amplitude * np.sin(x[b] * 2 * np.pi / wavelength)
T[c] = 0

In [ ]:
# Calculate numerical solution
iter = 0
while iter * dt < run_duration:
    iter += 1
    if v > 0:
        T[1:] -= v * dt / dx * np.diff(T)
        T[0] = T[-1]

    if iter % 10 == 0:
        plt.plot(x / 1.0e3, T, "b")
        plt.xlim((0, max(x / 1e3)))
        plt.xlabel("x (km)")
        plt.ylabel("T (K)")
        plt.title("Advection of Temperature Perturbation")
        plt.show()

### Downwind solution

Now, set the wind speed $v$ to -10 m/s and calculate the advection equation for negative velocities using Eq. 4.
Copy-paste your upwind solution and add the negative component to it.
Don't forget to reset the temperature array before you start. 

In [ ]:
# Set parameter values
v = -10

In [ ]:
# Initialize temperature array
T = np.zeros_like(x)
T[b] = amplitude * np.sin(x[b] * 2 * np.pi / wavelength)
T[c] = 0

In [ ]:
# Calculate numerical solution
iter = 0
while iter * dt < run_duration:
    iter += 1
    if v < 0:
        T[:-1] -= v * dt / dx * np.diff(T)
        T[-1] = T[0]

    if iter % 10 == 0:
        plt.plot(x / 1.0e3, T, "r")
        plt.xlim((0, max(x / 1e3)))
        plt.xlabel("x (km)")
        plt.ylabel("T (K)")
        plt.title("Advection of Temperature Perturbation")
        plt.show()

### Analytical solution

Can you find an analytical solution for this problem? Implement eq. 2. You need just one line of code (HINT use the `np.mod` statement). Plot and compare with the numerical solution. What is going on here. What can we do to resolve this issue? 

In [ ]:
# Initialize temperature array
T = np.zeros_like(x)
T[b] = amplitude * np.sin(x[b] * 2 * np.pi / wavelength)
T[c] = 0

In [ ]:
# Calculate numerical solution
for t in range(int(run_duration)):
    if v * t % dx == 0:
        T = np.roll(T, 1)

    if 0.25 * t % dx == 0:
        plt.plot(x / 1.0e3, T, "g")
        plt.xlim((0, max(x / 1e3)))
        plt.xlabel("x (km)")
        plt.ylabel("T (K)")
        plt.title("Advection of Temperature Perturbation")
        plt.show()

## Practice your skills: Eyjafjallajokull- Part 2

Back to the problem of the Eyjafjallajokull volcano. This time we will solve the advection diffusion equation in 1D: 

$$\frac{\partial C}{\partial t} = -v \frac{\partial C}{\partial x} + D\frac{\delta^2 C}{\delta x^2} 
\label{eq:5}\tag{5}$$

Copy-paste your script from the [diffusion](./diffusion.ipynb) notebook. 
Adjust the script by adding advection.
You can use the code derived in the previous blocks. 

You only need one additional model parameter: the advection velocity $v$. 
- Assume an advection velocity of 10 km/h (that is, the wind speed at which ash aerosols are advected towards the continent)
- Calculate the time step by combining the CFL criterium for advection (see above) and diffusion:
~~~
dt_a = 0.9*dx/v
dt_d = dx*dx/D/2.5
dt = min(dt_a,dt_d)
print('dt is: ' + str(dt) + 'hours')
~~~
- Same question: after how many hours do we get 5 ppm ash aerosols in Brussels?

Enter the parameter values and initialize model variables.

In [ ]:
# physics
D = 25
Lx = 5000
time = 0
v = 10

# numerical properties
dx = 20
x = np.arange(start=0, stop=Lx, step=dx)
nx = len(x)
nt = 10000
nout = 1000

# Location of volcano and Brussels
ind_vol = int(2220 / dx)
ind_Bru = int(4220 / dx)

C_ini = 100
C_rate = 100
Cstart = 0
Cend = 0
C = np.zeros(x.shape)

C[0] = Cstart
C[ind_vol] = C_ini
C[-1] = Cend

plt.figure()
plt.plot(x, C)
plt.scatter(x[ind_Bru], C[ind_Bru], c="r")

# Calculate a stable time step
dt_a = 0.9 * dx / v
dt_d = dx * dx / D / 2.5
dt = min(dt_a, dt_d)
print("dt is: " + str(dt) + "hours")

Code up the while loop. 
The structure should look like this:
~~~
iter = 0 
while ...
    iter += 1

    # Advection 
    if v >= 0:
        ...       
    elif v < 0:
        ...

    # Boundary conditions
    C[0] = ...
    C[-1] = ...    
        
    # Diffusion 
    ...

    # Source term
    C[ind_vol] = ...
    

    if iter % 100 == 0:        
        plt.plot(x, C)
        plt.scatter(x[ind_Bru], C[ind_Bru], c='r')
        plt.title('Time is: ' + str(iter*dt) + ' hours')    
        plt.show()

print('Concentration reached after: ' + str(int(iter*dt)) + ' hours')
print('or : ' + str(int(iter*dt/24)) + ' days')
~~~


In [ ]:
iter = 0
while C[ind_Bru] < 5:
    iter += 1

    # Advection
    if v >= 0:
        C[1:] -= v * dt / dx * np.diff(C)
    else:
        C[:-1] -= v * dt / dx * np.diff(C)

    # Diffusion
    q = -D * np.diff(C) / dx
    C[1:-1] = C[1:-1] - dt * np.diff(q) / dx

    # Source term
    C[ind_vol] += C_rate * dt

    # Boundary conditions
    C[0] = Cstart
    C[-1] = Cend

    if iter % 100 == 0:
        plt.plot(x, C)
        plt.scatter(x[ind_Bru], C[ind_Bru], c="r")
        plt.title("Time is: " + str(iter * dt) + " hours")
        plt.show()

print("Concentration reached after: " + str(int(iter * dt)) + " hours")
print("or : " + str(int(iter * dt / 24)) + " days")

That's more like it! Congratulations, you have reached the end of this notebook!